In [ ]:
print("Spark session is Loaded !!!")
sdf = spark.read.csv('hdfs://ec2-18-223-23-172.us-east-2.compute.amazonaws.com:8020/user/root/temp.csv')
for i in range(0,2500):
    #sdf = sdf.union(sdf)
    #sdf.write.csv(path="large_temp_2.csv")
    sdf.write.save(path="large_temp_2.csv", format='csv', mode='append')

VBox()

In [ ]:
new_sdf = spark.read.csv("large_temp_2.csv",header=True,encoding='utf-8')
new_sdf.count()

##### New approach:

In [ ]:
def doubleIT():
    sdf = spark.read.csv("large_temp_2.csv")
    sdf.write.save(path="large_temp_2.csv", format='csv', mode='append')
    
for i in range(0,10):
    doubleIT()
    
    
sdf = spark.read.csv("large_temp_2.csv")
sdf.count()

* How to know the large_temp_2.csv FOLDER size?
    * **Ans**: hadoop fs -du -s -h /user/livy/large_temp_2.csv
		* 34.2 M  /user/livy/large_temp_2.csv
            * from the new approach of doubleIT, it should reach a TB atleast
    *  hdfs dfs -df -h
    ```
        Filesystem                                                  Size     Used  Available  Use%
        hdfs://ip-172-31-39-242.us-east-2.compute.internal:8020  497.8 G  182.5 G    297.6 G   37%
    ```

* can 2 jobs, one for append and one for count work in parallel ? 
    * when I tried 2 spark submit (from 2 cells) in HUE notebook, both spark job started together but job with "count" stopped till "append" job completed.
    
    
* what is time to read and count for csv file?
* what is time to read and count for parquete file?
* can panda hold the csv or parquet file ?
    * if yes, can it operate on a "query" with the same speed as that of spark dataframe?
        * obviously not, otherwise spark should have closed its shutter !!! but let us see the difference and hail spark
    * if no, can we slice the dataset using spark df and may be panda simply does the plot( then why not matplot lib?)
